# Balance reaction mass and charge

### Find reactions that have unbalanced charges, and add charges or protons etc. as necessary to repair.

Use list of unbalanced reactions with R functions in "investigate_metabolite_charges.r"

### TO DO: handle exchange reactions separately

Example scripts that include addressing unbalanced reactions:
https://github.com/ChristianLieven/memote-m-capsulatus/blob/master/ComplementaryScripts/12.%20ApplyMemoteDrivenCuration.ipynb
https://github.com/SysBioChalmers/Sco-GEM/blob/devel/code/curation/v1_3_2.py
https://github.com/SBRG/bigg_models/blob/master/bigg_models/tests/test_loaded_data.py
This process is especially relevant for carveme models: https://github.com/cdanielmachado/carveme/issues/136

Could also use alongside custom reaction databases:  https://github.com/SysBioChalmers/Salb-GEM/tree/master/ComplementaryData/curation

Import libraries, read in SBML v3 model

In [1]:
import cobra
import os
import numpy as np
import pandas as pd
import memote.support.consistency as consistency
import memote.support.consistency_helpers as con_helpers
from memote.utils import annotate, get_ids, truncate, wrapper
from six import iteritems, itervalues
from collections import defaultdict
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
model_in = cobra.io.read_sbml_model('/projectnb2/talbot-lab-data/metabolic_models/genomes/solibacter/solibacter.xml')

Using license file /usr/local/gurobi/gurobi.lic
Set parameter TokenServer to value sccsvc


Identify problematic reactions and their metaNET listings from annotations (https://github.com/opencobra/cobrapy/issues/684)

In [3]:
#rxn_list = consistency.find_mass_unbalanced_reactions(model_in.reactions)
rxn_list = consistency.find_charge_unbalanced_reactions(model_in.reactions) # handle mass & charge simultaneously?
print(rxn_list)


[<Reaction 2AGPGAT160 at 0x2af9638adfd0>, <Reaction 2AGPGAT161 at 0x2af9638adf98>, <Reaction 2DDARAA at 0x2af9638bebe0>, <Reaction 2MBALDH at 0x2af9638cabe0>, <Reaction 2S6HCCi at 0x2af9638d20f0>, <Reaction 3HAACOAT80 at 0x2af9638d8be0>, <Reaction A6PAG at 0x2af9638b7470>, <Reaction AACPS2 at 0x2af9638d8390>, <Reaction AACPS4 at 0x2af9638d8908>, <Reaction AACPS7 at 0x2af9638e3908>, <Reaction AACPS9 at 0x2af9638ec358>, <Reaction AALDH at 0x2af9638d8cf8>, <Reaction ACACT5r_1 at 0x2af9639446d8>, <Reaction ACACT6r_1 at 0x2af963944cc0>, <Reaction ACGAMPM at 0x2af96396c438>, <Reaction ACHBS at 0x2af96396c400>, <Reaction ACKILE at 0x2af963973278>, <Reaction ACLS_a at 0x2af96397b668>, <Reaction ACLS_b at 0x2af963985198>, <Reaction ACLS_d at 0x2af963985470>, <Reaction ACLSa_1 at 0x2af963985cc0>, <Reaction ACLSb_1 at 0x2af96398c0f0>, <Reaction ACM6PH at 0x2af96398c0b8>, <Reaction ACOAD3 at 0x2af9639a74e0>, <Reaction ACOAD3f at 0x2af9639a79b0>, <Reaction ACOAD4_1 at 0x2af9639a70f0>, <Reaction ACO

{'sbo': 'SBO:0000176', 'metanetx.reaction': 'MNXR94775'}

Fix mass and charge balances for one reaction (testing)

In [18]:
import memote.support.annotation as annotation

    
overview = annotation.generate_component_annotation_overview(rxn_list, "metanet")

#for rxn in rxn_list:
rxn = rxn_list[0]
rxn.check_mass_balance()
# Print metanetxID if exists
metanetx_id = rxn.annotation['metanetx.reaction']
print(metanetx_id)
print(rxn)

charge = 0
for metabolite, coefficient in iteritems(rxn.metabolites):
    print(metabolite)
    print(metabolite.charge)
    charge += coefficient * metabolite.charge

{'charge': 1.0}

MNXR94774
2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c
2agpg160_c
-1
atp_c
-4
hdca_c
-1
amp_c
-2
pg160_c
0
ppi_c
-3


Compare this reaction against the output from R script:

MetanetX has the following charges on left of equation: 
    hdca        h 2agpg160      atp 
      -1        1       -1       -4 
MetanetX has the following charges on right of equation: 
  ppi   amp pg160 
   -3    -2     0 
   
 We see that adding a proton to the left side of the equation should balance it.

In [21]:
rxn.add_metabolites({rxn.metabolites.h_c:1}, combine= True)
rxn.check_mass_balance()

#model_in.reactions.2AGPGAT160.add_metabolites({model.model_in.h_p:1}, combine= True)


AttributeError: 'dict' object has no attribute 'h_c'

In [84]:
model_in.metabolites.pg160_c.charge = -1
#model_in.metabolites.2ddara_c.charge = -1
rxn.metabolites

for metabolite, coefficient in iteritems(rxn.metabolites):
    rxn.metabolites.gcald_c.charge
    
rxn.check_mass_balance()



{<Metabolite 2ddara_c at 0x2b4d9e53f7f0>: -1.0,
 <Metabolite gcald_c at 0x2b4d9e767898>: 1.0,
 <Metabolite pyr_c at 0x2b4d9e9595f8>: 1.0}

AttributeError: 'dict' object has no attribute 'gcald_c'

In [49]:
    
#rxn.add_metabolites({model.metabolites.`2agpg161_c`:1}, combine= True)
rxn.check_mass_balance()
print(con_helpers.is_charge_balanced(rxn))

{}

True


### Run loop on all reactions

Apply to all problematic reactions and evaluate success

In [8]:
for rxn in rxn_list:
    print(rxn)
    #charge = model_in.reactions.rxn.check_mass_balance()
    #print(charge)

    charge = 0
    for metabolite, coefficient in iteritems(rxn.metabolites):
        charge += coefficient * metabolite.charge
    print(charge)
    print(con_helpers.is_charge_balanced(rxn))
    #model.reactions.PAMMOipp.add_metabolites({model.metabolites.h_p:1}, combine= True)

    

2AGPGAT160: 2agpg160_c + atp_c + hdca_c --> amp_c + pg160_c + ppi_c
1.0
False
2AGPGAT161: 2agpg161_c + atp_c + hdcea_c --> amp_c + pg161_c + ppi_c
-1.0
False
2DDARAA: 2ddara_c <=> gcald_c + pyr_c
-1.0
False
2MBALDH: 2mbald_c + h2o_c + nad_c <=> 2mba_c + 2.0 h_c + nadh_c
1.0
False
2S6HCCi: akg_c + h_c + ichor_c <=> 2shchc_c + co2_c + pyr_c
2.0
False
3HAACOAT80: 3hoctACP_c + coa_c <=> ACP_c + R_3hocoa_c
6.0
False
A6PAG: atp_c + gal_c --> adp_c + dgal6p_c + h_c
2.0
False
AACPS2: ACP_c + atp_c + ttdcea_c --> amp_c + ppi_c + tdeACP_c
-2.0
False
AACPS4: ACP_c + atp_c + hdcea_c --> amp_c + hdeACP_c + ppi_c
-2.0
False
AACPS7: ACP_c + atp_c + ddca_c --> amp_c + ddcaACP_c + ppi_c
-1.0
False
AACPS9: ACP_c + atp_c + octa_c --> amp_c + ocACP_c + ppi_c
-2.0
False
AALDH: nadh_c + pacald_c <=> nad_c + pea_c
1.0
False
ACACT5r_1: accoa_c + dccoa_c <=> 3oddcoa_c + coa_c
-4.0
False
ACACT6r_1: accoa_c + ddcoa_c <=> 3otdcoa_c + coa_c
-4.0
False
ACGAMPM: acgam6p_c <=> acgam1p_c
2.0
False
ACHBS: 2obut_c + h_c

In [ ]:
for rxn in rxn_list:
    print(rxn)
    balance = defaultdict(int)
    for metabolite, coefficient in iteritems(rxn.metabolites):
        for element, amount in iteritems(metabolite.elements):
            balance[element] += coefficient * amount
    #print(balance.values)
    h_val = balance.pop('H')
    print(h_val)
    balance = rxn.check_mass_balance()
    print(balance)
    rxn.add_metabolites({model.model_in.h_p:-1}, combine= True)
    rxn.check_mass_balance()

    #if h_val == 1:
    #    rxn.add_metabolites({model.metabolites.h_p:1}, combine= True)
    
#model_in.reactions.SALCHS4FEabcpp.check_mass_balance()
#model.reactions.NO3t2pp.add_metabolites({model.metabolites.h_p:1}, combine= True)
    
    

In [11]:
from importlib_resources import open_text
import memote.support.data

with open_text(
    memote.support.data, "met_id_shortlist.json", encoding="utf-8"
) as file_handle:
    METANETX_SHORTLIST = pd.read_json(file_handle)